In [ ]:
# export
from nbdev.imports import *
from nbdev.sync import *
from nbdev.export import *

In [ ]:
# default_exp test
# all_tst

# Extract tests

> The functions that grab the cells containing tests (filtering with potential flags) and execute them

Everything that is not an exported cell is considered a test, so you should make sure your notebooks can all run smoothly (and fast) if you want to use this functionality as the CLI. You can mark some cells with special flags (like slow) to make sure they are only executed when you authorize it. Those flags should be configured in your `settings.ini` (separated by a `|` if you have several of them). You can also apply a flag to one entire notebook by putting `# all_flag` in one of its cells.

## Detect flags

The following functions detect the cells that should be excluded from the tests (unless their special flag is passed).

In [ ]:
# export
_re_all_flag = re.compile("""
# Matches any line with #all_something and catches that something in a group:
^         # beginning of line (since re.MULTILINE is passed)
\s*       # any number of whitespace
\#\s*     # # then any number of whitespace
all_(\S+) # all_ followed by a group with any non-whitespace chars
\s*       # any number of whitespace
$         # end of line (since re.MULTILINE is passed)
""", re.IGNORECASE | re.MULTILINE | re.VERBOSE)

In [ ]:
# export
def check_all_flag(cells):
    "Check for an `# all_flag` cell and then return said flag"
    for cell in cells:
        if check_re(cell, _re_all_flag): return check_re(cell, _re_all_flag).groups()[0]

In [ ]:
nb = read_nb("04_test.ipynb")
test_eq(check_all_flag(nb['cells']), 'tst')
nb = read_nb("00_export.ipynb")
assert check_all_flag(nb['cells']) is None

In [ ]:
# export
_re_flags = re.compile(f"""
# Matches any line with a test flad and catches it in a group:
^               # beginning of line (since re.MULTILINE is passed)
\s*             # any number of whitespace
\#\s*           # # then any number of whitespace
({Config().get('tst_flags', '')})
\s*             # any number of whitespace
$               # end of line (since re.MULTILINE is passed)
""", re.IGNORECASE | re.MULTILINE | re.VERBOSE)

In [ ]:
# export
def get_cell_flags(cell):
    "Check for any special test flag in `cell`"
    if cell['cell_type'] != 'code' or len(Config().get('tst_flags',''))==0: return []
    return _re_flags.findall(cell['source'])

In [ ]:
test_eq(get_cell_flags({'cell_type': 'code', 'source': "#hide\n# tst\n"}), ['tst'])
test_eq(get_cell_flags({'cell_type': 'code', 'source': "#hide\n"}), [])

## Testing a notebook

In [ ]:
# export
def _add_import_cell(mod):
    "Return an import cell for `mod`"
    return {'cell_type': 'code',
            'execution_count': None,
            'metadata': {'hide_input': True},
            'outputs': [],
            'source': f"\nfrom local.{mod} import *"}

In [ ]:
# export
_re_is_export = re.compile(r"""
# Matches any text with #export or #exports flag:
^         # beginning of line (since re.MULTILINE is passed)
\s*       # any number of whitespace
\#\s*     # # then any number of whitespace
exports?  # export or exports
\s*       # any number of whitespace
""", re.IGNORECASE | re.MULTILINE | re.VERBOSE)

In [ ]:
# export
_re_has_import = re.compile(r"""
# Matches any text with import statement:
^         # beginning of line (since re.MULTILINE is passed)
\s*       # any number of whitespace
import    # # then any number of whitespace
\s+
|
\s*
from
\s+\S+\s+
import
\s+
""", re.IGNORECASE | re.MULTILINE | re.VERBOSE)

In [ ]:
# export
_re_show_doc = re.compile(r"""
# Matches any text with show_doc(...)
^            # beginning of line (since re.MULTILINE is passed)
\s*          # any number of whitespace
show_doc\(     # # then any number of whitespace
[^\)]*  # export or exports
\)\s*$       # any number of whitespace
""", re.IGNORECASE | re.MULTILINE | re.VERBOSE)

In [ ]:
# export
def _add_cell(cell, flags):
    if cell['cell_type']!='code': return False
    if _re_show_doc.search(cell['source']) is not None: return False
    for f in get_cell_flags(cell):
        if f not in flags: return False
    return True

In [ ]:
# export
def export_nb(nb, tmp_f, flags=None):
    "Export the tests that don't have a flag in `flags` in `nb` to `tmp_f`"
    if flags is None: flags = []
    all_flag = check_all_flag(nb['cells'])
    if all_flag is not None and all_flag not in flags: return
    for cell in nb['cells']:
        if not _add_cell(cell, flags): continue
        code = f'{cell["source"]}\n\n'
        with open(tmp_f, 'a', encoding='utf8') as f: f.write(code)

In [ ]:
fname = Path.cwd()/"00_export.ipynb"
tmp_f = Path('tst')/(fname.with_suffix('.py')).name
if tmp_f.exists(): tmp_f.unlink()
export_nb(read_nb(fname), Path('tst')/(fname.with_suffix('.py')).name)

In [ ]:
import runpy

In [ ]:
# export
def test_nb(fn, flags=None, tmp_dir='.'):
    "Execute notebook in `fn` with `flags` inside `tmp_dir`"
    fn = Path(fn)
    tmp_f = Path(tmp_dir).absolute()/(fn.with_suffix('.py')).name
    if tmp_f.exists(): tmp_f.unlink()
    os.environ["IN_TEST"] = '1'
    try:
        nb = read_nb(fn)
        export_nb(nb, tmp_f, flags=flags)
        runpy.run_path(tmp_f)
    except Exception as e: print(e)
    finally: 
        #if tmp_f.exists(): tmp_f.unlink()
        os.environ.pop("IN_TEST")

In [ ]:
test_nb("00_export.ipynb")

## Export-

In [ ]:
#hide
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_merge.ipynb.
Converted 06_cli.ipynb.
Converted 99_search.ipynb.
Converted index.ipynb.
